In [1]:
%run _standard_imports.ipynb
from pyfasta import Fasta
from Bio.Seq import Seq

python 3.5.2 |Continuum Analytics, Inc.| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
numpy 1.11.3
scipy 0.18.1
pandas 0.19.2
numexpr 2.6.2
pysam 0.8.4
petl 1.1.0
petlx 1.0.3
vcf 0.6.8
vcfnp 2.3.0
h5py 2.6.0
tables 3.3.0


In [39]:
scratch_dir = "/lustre/scratch118/malaria/team112/personal/rp7/data/methods-dev/builds/Pf6.0/20170303_run_Olivo_GRC"
output_dir = "/nfs/team112_internal/rp7/data/methods-dev/builds/Pf6.0/20170303_run_Olivo_GRC"
!mkdir -p {scratch_dir}/grc
!mkdir -p {scratch_dir}/grc_inc_unmapped
!mkdir -p {scratch_dir}/species
!mkdir -p {scratch_dir}/species_inc_unmapped
!mkdir -p {scratch_dir}/log
!mkdir -p {output_dir}/grc
!mkdir -p {output_dir}/grc_inc_unmapped
!mkdir -p {output_dir}/species
!mkdir -p {output_dir}/species_inc_unmapped

bam_fn = "%s/pf_60_mergelanes.txt" % output_dir
bam_list_fn = "%s/pf_60_mergelanes_bamfiles.txt" % output_dir
chromosomeMap_fn = "%s/chromosomeMap.tab" % output_dir

grc_properties_fn = "%s/grc/grc.properties" % output_dir
grc_inc_unmapped_properties_fn = "%s/grc_inc_unmapped/grc.properties" % output_dir
species_properties_fn = "%s/species/species.properties" % output_dir
species_inc_unmapped_properties_fn = "%s/species_inc_unmapped/species.properties" % output_dir

submitArray_fn = "%s/grc/submitArray.sh" % output_dir
submitIncUnmappedArray_fn = "%s/grc_inc_unmapped/submitArray.sh" % output_dir
submitSpeciesArray_fn = "%s/species/submitArray.sh" % output_dir
submitSpeciesIncUnmappedArray_fn = "%s/species_inc_unmapped/submitArray.sh" % output_dir

runArrayJob_fn = "%s/grc/runArrayJob.sh" % output_dir
runSpeciesArrayJob_fn = "%s/species/runArrayJob.sh" % output_dir

mergeGrcResults_fn = "%s/grc/mergeGrcResults.sh" % output_dir
mergeGrcIncUnmappedResults_fn = "%s/grc_inc_unmapped/mergeGrcResults.sh" % output_dir
mergeSpeciesResults_fn = "%s/species/mergeSpeciesResults.sh" % output_dir
mergeSpeciesIncUnmappedResults_fn = "%s/species_inc_unmapped/mergeSpeciesResults.sh" % output_dir

ref_fasta_fn = "%s/Pfalciparum.genome.fasta" % output_dir
# old_ref_fasta_fn = "%s/3D7_V3.fasta" % output_dir

# Update GRC code

In [3]:
current_dir = !pwd

In [4]:
current_dir[0]

'/nfs/team112_internal/rp7/src/github/malariagen/methods-dev/builds/Pf 6.0/notebooks'

In [5]:
%cd /nfs/team112_internal/rp7/src/github/malariagen/GeneticReportCard

/nfs/team112_internal/rp7/src/github/malariagen/GeneticReportCard


In [6]:
!git pull

Already up-to-date.


In [7]:
%cd SequencingReadsAnalysis

/nfs/team112_internal/rp7/src/github/malariagen/GeneticReportCard/SequencingReadsAnalysis


In [8]:
!ant build

Buildfile: /nfs/team112_internal/rp7/src/github/malariagen/GeneticReportCard/SequencingReadsAnalysis/build.xml

build-subprojects:

init:

build-project:
     [echo] AnalysisCommon: /nfs/team112_internal/rp7/src/github/malariagen/GeneticReportCard/AnalysisCommon/build.xml

init:

build-project:
     [echo] SequencingReadsAnalysis: /nfs/team112_internal/rp7/src/github/malariagen/GeneticReportCard/SequencingReadsAnalysis/build.xml

build:

BUILD SUCCESSFUL
Total time: 0 seconds


In [9]:
%cd {current_dir[0]}

/nfs/team112_internal/rp7/src/github/malariagen/methods-dev/builds/Pf 6.0/notebooks


# Gather relevant data

In [10]:
!cp /lustre/scratch116/malaria/pfalciparum/resources/Pfalciparum.genome.fasta {ref_fasta_fn}
# !cp /lustre/scratch109/malaria/pfalciparum/resources/3D7_V3.fasta {old_ref_fasta_fn}

In [12]:
# Create list of bam files in format required
tbl_bam_file = (etl
    .fromtsv(bam_fn)
    .addfield('ChrMap', 'default')
    .rename('path', 'BamFile')
    .rename('sample', 'Sample')
    .cut(['Sample', 'BamFile', 'ChrMap'])
    .select(lambda rec: 'pe' in rec['BamFile']) # There are 5 single-end bam files which should be ignored
    .addrownumbers()
    .sort('row', reverse=True) # we reverse it because the largest files are at the end of the list, and we want these kicked off first
    .cutout('row')
)
tbl_bam_file.totsv(bam_list_fn, lineterminator='\n')
# !dos2unix -o {bam_list_fn}

In [13]:
tbl_bam_file

Sample,BamFile,ChrMap
PA0867-CW,/lustre/scratch116/malaria/pfalciparum/output/6/d/b/0/37432/4_bam_mark_duplicates_v2/pe.1.markdup.bam,default
PG0035-Cx,/lustre/scratch116/malaria/pfalciparum/output/b/a/c/9/40053/4_bam_mark_duplicates_v2/pe.1.markdup.bam,default
PF0252-C,/lustre/scratch116/malaria/pfalciparum/output/9/9/9/6/39138/4_bam_mark_duplicates_v2/pe.1.markdup.bam,default
PG0528-C,/lustre/scratch116/malaria/pfalciparum/output/7/a/e/0/40666/4_bam_mark_duplicates_v2/pe.1.markdup.bam,default
PM0603-C,/lustre/scratch116/malaria/pfalciparum/output/0/5/c/7/42548/4_bam_mark_duplicates_v2/pe.1.markdup.bam,default


In [14]:
len(tbl_bam_file)

8247

In [15]:
len(tbl_bam_file.select(lambda rec: 'pe' in rec['BamFile']))

8247

In [16]:
!wc -l {bam_list_fn}

8247 /nfs/team112_internal/rp7/data/methods-dev/builds/Pf6.0/20170303_run_Olivo_GRC/pf_60_mergelanes_bamfiles.txt


# Create relevant GRC properties files

In [17]:
fo = open(grc_properties_fn, 'w')
print('''grc.loci=crt_core,crt_ex01,crt_ex02,crt_ex03,crt_ex04,crt_ex06,crt_ex09,crt_ex10,crt_ex11,dhfr_1,dhfr_2,dhfr_3,dhps_1,dhps_2,dhps_3,dhps_4,mdr1_1,mdr1_2,mdr1_3,arps10,mdr2,fd,exo

# CRT
grc.locus.crt_core.region=Pf3D7_07_v3:403500-403800
grc.locus.crt_core.targets=crt_72-76@403612-403626
grc.locus.crt_core.anchors=403593@TATTATTTATTTAAGTGTA,403627@ATTTTTGCTAAAAGAAC

grc.locus.crt_ex01.region=Pf3D7_07_v3:403150-404420
grc.locus.crt_ex01.targets=crt_24@403291-403293
grc.locus.crt_ex01.anchors=403273@GAGCGTTATA.[AG]GAATTA...AATTTA.TACAAGAA[GA]GAA

grc.locus.crt_ex02.region=Pf3D7_07_v3:403550-403820
grc.locus.crt_ex02.targets=crt_97@403687-403689
grc.locus.crt_ex02.anchors=403657@GGTAACTATAGTTTTGT.[AT]CATC[CT]GAAAC,403690@AACTTTATTTGTATGATTA[TA]GTTCTTTATT

grc.locus.crt_ex03.region=Pf3D7_07_v3:403850-404170
grc.locus.crt_ex03.targets=crt_144@404007-404009,crt_148@404019-404021
grc.locus.crt_ex03.anchors=404022@ACAAGAACTACTGGAAA[TC]AT[CT]CA[AG]TCATTT,403977@TC[CT]AT.TTA.AT[GT]CCTGTTCA.T[CA]ATT

grc.locus.crt_ex04.region=Pf3D7_07_v3:404200-404500
grc.locus.crt_ex04.targets=crt_194@404329-404331,crt_220@404407-404409
grc.locus.crt_ex04.anchors=404304@CGGAGCA[GC]TTATTATTGTTGTAACA...GCTC,404338@GTAGAAATGAAATTATC[TA]TTTGAAACAC,404359@GAAACACAAGAAGAAAATTCTATC[AG]TATTTAATC,404382@C[AG]TATTTAATCTTGTCTTA[AT]TTAGT...TTAATTG

grc.locus.crt_ex06.region=Pf3D7_07_v3:404700-405000
grc.locus.crt_ex06.targets=crt_271@404836-404838
grc.locus.crt_ex06.anchors=404796@TTGTCTTATATT.CCTGTATACACCCTTCCATT[TC]TTAAAA...C

grc.locus.crt_ex09.region=Pf3D7_07_v3:405200-405500
grc.locus.crt_ex09.targets=crt_326@405361-405363,crt_333@405382-405384
grc.locus.crt_ex09.anchors=405334@AAAACCTT[CT]G[CT]ATTGTTTTCCTTCTTT,405364@A.TTGTGATAATTTAATA...AGCTAT

grc.locus.crt_ex10.region=Pf3D7_07_v3:405400-405750
grc.locus.crt_ex10.targets=crt_342@405557-405559,crt_356@405599-405601
grc.locus.crt_ex10.anchors=405539@ATTATCGACAAATTTTCT...[AT]TGACATATAC,405573@TTGTTAGTTGTATACAAG[GT]TCCA[GA]CA,405602@GCAATT[GT]CTTATTACTTTAAATTCTTA[GA]CC

grc.locus.crt_ex11.region=Pf3D7_07_v3:405700-406000
grc.locus.crt_ex11.targets=crt_371@405837-405839
grc.locus.crt_ex11.anchors=405825@[GT]GTGATGTT.[TA]A...G.ACCAAGATTATTAG,405840@G.ACCAAGATTATTAGATTTCGTAACTTTG

# DHFR
grc.locus.dhfr_1.region=Pf3D7_04_v3:748100-748400
grc.locus.dhfr_1.targets=dhfr_51@748238-748240,dhfr_59@748262-748264
grc.locus.dhfr_1.anchors=748200@GAGGTCTAGGAAATAAAGGAGTATTACCATGGAA,748241@TCCCTAGATATGAAATATTTT...GCAG,748265@GCAGTTACAACATATGTGAATGAATC

grc.locus.dhfr_2.region=Pf3D7_04_v3:748250-748550
grc.locus.dhfr_2.targets=dhfr_108@748409-748411
grc.locus.dhfr_2.anchors=748382@CAAAATGTTGTAGTTATGGGAAGAACA,748412@TGGGAAAGCATTCCAAAAAAATTT

grc.locus.dhfr_3.region=Pf3D7_04_v3:748400-748720
grc.locus.dhfr_3.targets=dhfr_164@748577-748579
grc.locus.dhfr_3.anchors=748382@GGGAAATTAAATTACTATAAATG,748382@CTATAAATGTTTTATT...GGAGGTTC,748412@GGAGGTTCCGTTGTTTATCAAG


# DHPS
grc.locus.dhps_1.region=Pf3D7_08_v3:549550-549750
grc.locus.dhps_1.targets=dhps_436@549681-549683,dhps_437@549684-549686
grc.locus.dhps_1.anchors=549657@GTTATAGAT[AG]TAGGTGGAGAATCC,549669@GGTGGAGAATCC..TG.TCC,549687@CCTTTTGTTAT[AG]CCTAATCCAAAAATTAGTG

grc.locus.dhps_2.region=Pf3D7_08_v3:549850-550150
grc.locus.dhps_2.targets=dhps_540@549993-549995
grc.locus.dhps_2.anchors=549949@GTGTAGTTCTAATGCATAAAAGAGG,549970@GAGGAAATCCACATACAATGGAT,549985@CAATGGAT...CTAACAAATTA[TA]GATA,549996@CTAACAAATTA[TA]GATAATCTAGT

grc.locus.dhps_3.region=Pf3D7_08_v3:549950-550250
grc.locus.dhps_3.targets=dhps_581@550116-550118
grc.locus.dhps_3.anchors=550092@CTATTTGATATTGGATTAGGATTT,550119@AAGAAACATGATCAATCT[AT]TTAAACTC

grc.locus.dhps_4.region=Pf3D7_08_v3:550050-550350
grc.locus.dhps_4.targets=dhps_613@550212-550214
grc.locus.dhps_4.anchors=550167@GATGAGTATCCACTTTTTATTGG,550188@GGATATTCAAGAAAAAGATTTATT,550215@CATTGCATGAATGATCAAAATGTTG


# MDR1
grc.locus.mdr1_1.region=Pf3D7_05_v3:957970-958280
grc.locus.mdr1_1.targets=mdr1_86@958145-958147
grc.locus.mdr1_1.anchors=958120@GTTTG[GT]TGTAATATTAAA[GA]AACATG,958141@CATG...TTAGGTGATGATATTAATCCT

grc.locus.mdr1_2.region=Pf3D7_05_v3:958300-958600
grc.locus.mdr1_2.targets=mdr1_184@958439-958441
grc.locus.mdr1_2.anchors=958413@CATATGC[CA]AGTTCCTTTTTAGG,958446@GGTC[AG]TTAATAAAAAAT[GA]CACGTTTGAC

grc.locus.mdr1_3.region=Pf3D7_05_v3:961470-961770
grc.locus.mdr1_3.targets=mdr1_1246@961625-961627
grc.locus.mdr1_3.anchors=961595@GTTATAGAT[AG]TAGGTGGAGAATCC,961628@CTTAGAAA[CT][TA]TATTTTC[AT]ATAGTTAGTC

# ARPS10
grc.locus.arps10.region=Pf3D7_14_v3:2480900-2481200
grc.locus.arps10.targets=arps10_127@2481070-2481072
grc.locus.arps10.anchors=2481045@ATTTAC[CA]TTTTTGCGATCTCCCCAT...[GC],2481079@GACAGT[AC]G[AG]GA[GA]CAATTCGAAATAAAAC

# MDR2
grc.locus.mdr2.region=Pf3D7_14_v3:1956070-1956370
grc.locus.mdr2.targets=mdr2_484@-1956224-1956226
grc.locus.mdr2.anchors=1956203@ACATGTTATTAATCCT[TC]TAT...TGCC,1956227@TGCCGGAATAAT[AG]TACATTAAAACAGAAC

# Ferredoxin
grc.locus.fd.region=Pf3D7_13_v3:748250-748550
grc.locus.fd.targets=fd_193@-748393-748395
grc.locus.fd.anchors=748396@[GA]TGTAGTTCGTCTTCCTTGTG[CT]GTTTC

# Exo
grc.locus.exo.region=Pf3D7_13_v3:2504400-2504700
grc.locus.exo.targets=exo_415@2504559-2504561
grc.locus.exo.anchors=2504526@[GC]ATGATTTTA[AG][CA]AATATGGT[TC]ATAA[CT]GATAAAA,2504562@GAA[GT]TAAA[CT][AC]ATCATTGG[GA]AAAA[TC]AATATATAC
''', file=fo)
fo.close() 

In [40]:
fo = open(grc_inc_unmapped_properties_fn, 'w')
print('''grc.loci=crt_core,crt_ex01,crt_ex02,crt_ex03,crt_ex04,crt_ex06,crt_ex09,crt_ex10,crt_ex11,dhfr_1,dhfr_2,dhfr_3,dhps_1,dhps_2,dhps_3,dhps_4,mdr1_1,mdr1_2,mdr1_3,arps10,mdr2,fd,exo

# CRT
grc.locus.crt_core.region=Pf3D7_07_v3:403500-403800
grc.locus.crt_core.targets=crt_72-76@403612-403626
grc.locus.crt_core.anchors=403593@TATTATTTATTTAAGTGTA,403627@ATTTTTGCTAAAAGAAC
grc.locus.crt_core.analyzeUnmappedReads=true

grc.locus.crt_ex01.region=Pf3D7_07_v3:403150-404420
grc.locus.crt_ex01.targets=crt_24@403291-403293
grc.locus.crt_ex01.anchors=403273@GAGCGTTATA.[AG]GAATTA...AATTTA.TACAAGAA[GA]GAA
grc.locus.crt_ex01.analyzeUnmappedReads=true

grc.locus.crt_ex02.region=Pf3D7_07_v3:403550-403820
grc.locus.crt_ex02.targets=crt_97@403687-403689
grc.locus.crt_ex02.anchors=403657@GGTAACTATAGTTTTGT.[AT]CATC[CT]GAAAC,403690@AACTTTATTTGTATGATTA[TA]GTTCTTTATT
grc.locus.crt_ex02.analyzeUnmappedReads=true

grc.locus.crt_ex03.region=Pf3D7_07_v3:403850-404170
grc.locus.crt_ex03.targets=crt_144@404007-404009,crt_148@404019-404021
grc.locus.crt_ex03.anchors=404022@ACAAGAACTACTGGAAA[TC]AT[CT]CA[AG]TCATTT,403977@TC[CT]AT.TTA.AT[GT]CCTGTTCA.T[CA]ATT
grc.locus.crt_ex03.analyzeUnmappedReads=true

grc.locus.crt_ex04.region=Pf3D7_07_v3:404200-404500
grc.locus.crt_ex04.targets=crt_194@404329-404331,crt_220@404407-404409
grc.locus.crt_ex04.anchors=404304@CGGAGCA[GC]TTATTATTGTTGTAACA...GCTC,404338@GTAGAAATGAAATTATC[TA]TTTGAAACAC,404359@GAAACACAAGAAGAAAATTCTATC[AG]TATTTAATC,404382@C[AG]TATTTAATCTTGTCTTA[AT]TTAGT...TTAATTG
grc.locus.crt_ex04.analyzeUnmappedReads=true

grc.locus.crt_ex06.region=Pf3D7_07_v3:404700-405000
grc.locus.crt_ex06.targets=crt_271@404836-404838
grc.locus.crt_ex06.anchors=404796@TTGTCTTATATT.CCTGTATACACCCTTCCATT[TC]TTAAAA...C
grc.locus.crt_ex06.analyzeUnmappedReads=true

grc.locus.crt_ex09.region=Pf3D7_07_v3:405200-405500
grc.locus.crt_ex09.targets=crt_326@405361-405363,crt_333@405382-405384
grc.locus.crt_ex09.anchors=405334@AAAACCTT[CT]G[CT]ATTGTTTTCCTTCTTT,405364@A.TTGTGATAATTTAATA...AGCTAT
grc.locus.crt_ex09.analyzeUnmappedReads=true

grc.locus.crt_ex10.region=Pf3D7_07_v3:405400-405750
grc.locus.crt_ex10.targets=crt_342@405557-405559,crt_356@405599-405601
grc.locus.crt_ex10.anchors=405539@ATTATCGACAAATTTTCT...[AT]TGACATATAC,405573@TTGTTAGTTGTATACAAG[GT]TCCA[GA]CA,405602@GCAATT[GT]CTTATTACTTTAAATTCTTA[GA]CC
grc.locus.crt_ex10.analyzeUnmappedReads=true

grc.locus.crt_ex11.region=Pf3D7_07_v3:405700-406000
grc.locus.crt_ex11.targets=crt_371@405837-405839
grc.locus.crt_ex11.anchors=405825@[GT]GTGATGTT.[TA]A...G.ACCAAGATTATTAG,405840@G.ACCAAGATTATTAGATTTCGTAACTTTG
grc.locus.crt_ex11.analyzeUnmappedReads=true

# DHFR
grc.locus.dhfr_1.region=Pf3D7_04_v3:748100-748400
grc.locus.dhfr_1.targets=dhfr_51@748238-748240,dhfr_59@748262-748264
grc.locus.dhfr_1.anchors=748200@GAGGTCTAGGAAATAAAGGAGTATTACCATGGAA,748241@TCCCTAGATATGAAATATTTT...GCAG,748265@GCAGTTACAACATATGTGAATGAATC
grc.locus.dhfr_1.analyzeUnmappedReads=true

grc.locus.dhfr_2.region=Pf3D7_04_v3:748250-748550
grc.locus.dhfr_2.targets=dhfr_108@748409-748411
grc.locus.dhfr_2.anchors=748382@CAAAATGTTGTAGTTATGGGAAGAACA,748412@TGGGAAAGCATTCCAAAAAAATTT
grc.locus.dhfr_2.analyzeUnmappedReads=true

grc.locus.dhfr_3.region=Pf3D7_04_v3:748400-748720
grc.locus.dhfr_3.targets=dhfr_164@748577-748579
grc.locus.dhfr_3.anchors=748382@GGGAAATTAAATTACTATAAATG,748382@CTATAAATGTTTTATT...GGAGGTTC,748412@GGAGGTTCCGTTGTTTATCAAG
grc.locus.dhfr_3.analyzeUnmappedReads=true


# DHPS
grc.locus.dhps_1.region=Pf3D7_08_v3:549550-549750
grc.locus.dhps_1.targets=dhps_436@549681-549683,dhps_437@549684-549686
grc.locus.dhps_1.anchors=549657@GTTATAGAT[AG]TAGGTGGAGAATCC,549669@GGTGGAGAATCC..TG.TCC,549687@CCTTTTGTTAT[AG]CCTAATCCAAAAATTAGTG
grc.locus.dhps_1.analyzeUnmappedReads=true

grc.locus.dhps_2.region=Pf3D7_08_v3:549850-550150
grc.locus.dhps_2.targets=dhps_540@549993-549995
grc.locus.dhps_2.anchors=549949@GTGTAGTTCTAATGCATAAAAGAGG,549970@GAGGAAATCCACATACAATGGAT,549985@CAATGGAT...CTAACAAATTA[TA]GATA,549996@CTAACAAATTA[TA]GATAATCTAGT
grc.locus.dhps_2.analyzeUnmappedReads=true

grc.locus.dhps_3.region=Pf3D7_08_v3:549950-550250
grc.locus.dhps_3.targets=dhps_581@550116-550118
grc.locus.dhps_3.anchors=550092@CTATTTGATATTGGATTAGGATTT,550119@AAGAAACATGATCAATCT[AT]TTAAACTC
grc.locus.dhps_3.analyzeUnmappedReads=true

grc.locus.dhps_4.region=Pf3D7_08_v3:550050-550350
grc.locus.dhps_4.targets=dhps_613@550212-550214
grc.locus.dhps_4.anchors=550167@GATGAGTATCCACTTTTTATTGG,550188@GGATATTCAAGAAAAAGATTTATT,550215@CATTGCATGAATGATCAAAATGTTG
grc.locus.dhps_4.analyzeUnmappedReads=true


# MDR1
grc.locus.mdr1_1.region=Pf3D7_05_v3:957970-958280
grc.locus.mdr1_1.targets=mdr1_86@958145-958147
grc.locus.mdr1_1.anchors=958120@GTTTG[GT]TGTAATATTAAA[GA]AACATG,958141@CATG...TTAGGTGATGATATTAATCCT
grc.locus.mdr1_1.analyzeUnmappedReads=true

grc.locus.mdr1_2.region=Pf3D7_05_v3:958300-958600
grc.locus.mdr1_2.targets=mdr1_184@958439-958441
grc.locus.mdr1_2.anchors=958413@CATATGC[CA]AGTTCCTTTTTAGG,958446@GGTC[AG]TTAATAAAAAAT[GA]CACGTTTGAC
grc.locus.mdr1_2.analyzeUnmappedReads=true

grc.locus.mdr1_3.region=Pf3D7_05_v3:961470-961770
grc.locus.mdr1_3.targets=mdr1_1246@961625-961627
grc.locus.mdr1_3.anchors=961595@GTTATAGAT[AG]TAGGTGGAGAATCC,961628@CTTAGAAA[CT][TA]TATTTTC[AT]ATAGTTAGTC
grc.locus.mdr1_3.analyzeUnmappedReads=true

# ARPS10
grc.locus.arps10.region=Pf3D7_14_v3:2480900-2481200
grc.locus.arps10.targets=arps10_127@2481070-2481072
grc.locus.arps10.anchors=2481045@ATTTAC[CA]TTTTTGCGATCTCCCCAT...[GC],2481079@GACAGT[AC]G[AG]GA[GA]CAATTCGAAATAAAAC
grc.locus.arps10.analyzeUnmappedReads=true

# MDR2
grc.locus.mdr2.region=Pf3D7_14_v3:1956070-1956370
grc.locus.mdr2.targets=mdr2_484@-1956224-1956226
grc.locus.mdr2.anchors=1956203@ACATGTTATTAATCCT[TC]TAT...TGCC,1956227@TGCCGGAATAAT[AG]TACATTAAAACAGAAC
grc.locus.mdr2.analyzeUnmappedReads=true

# Ferredoxin
grc.locus.fd.region=Pf3D7_13_v3:748250-748550
grc.locus.fd.targets=fd_193@-748393-748395
grc.locus.fd.anchors=748396@[GA]TGTAGTTCGTCTTCCTTGTG[CT]GTTTC
grc.locus.fd.analyzeUnmappedReads=true

# Exo
grc.locus.exo.region=Pf3D7_13_v3:2504400-2504700
grc.locus.exo.targets=exo_415@2504559-2504561
grc.locus.exo.anchors=2504526@[GC]ATGATTTTA[AG][CA]AATATGGT[TC]ATAA[CT]GATAAAA,2504562@GAA[GT]TAAA[CT][AC]ATCATTGG[GA]AAAA[TC]AATATATAC
grc.locus.exo.analyzeUnmappedReads=true
''', file=fo)
fo.close() 

In [18]:
# Sanity check anchors for MDR2 and FD
print(str(Seq(Fasta(ref_fasta_fn)['Pf3D7_14_v3'][2481044:2481073])).upper()) # ARPS10
print(str(Seq(Fasta(ref_fasta_fn)['Pf3D7_14_v3'][2481078:2481106])).upper()) # ARPS10
print()
print(str(Seq(Fasta(ref_fasta_fn)['Pf3D7_14_v3'][1956202:1956230])).upper()) # MDR2
print(str(Seq(Fasta(ref_fasta_fn)['Pf3D7_14_v3'][1956226:1956255])).upper()) # MDR2
print()
print(str(Seq(Fasta(ref_fasta_fn)['Pf3D7_13_v3'][748395:748422])).upper()) # Ferredoxin


ATTTACCTTTTTGCGATCTCCCCATGTGG
GACAGTAGAGAGCAATTCGAAATAAAAC

ACATGTTATTAATCCTTCTATTGTTGCC
TGCCGGAATAATATACATTAAAACAGAAC

GTGTAGTTCGTCTTCCTTGTGCGTTTC


In [19]:
fo = open(species_properties_fn, 'w')
print('''sampleClass.classes=Pf,Pv,Pm,Pow,Poc,Pk
sampleClass.loci=mito1,mito2,mito3,mito4,mito5,mito6 

sampleClass.locus.mito1.region=Pf_M76611:520-820 
sampleClass.locus.mito1.anchors=651@CCTTACGTACTCTAGCT....ACACAA
sampleClass.locus.mito1.targets=species1@668-671&678-683
sampleClass.locus.mito1.target.species1.alleles=Pf@ATGATTGTCT|ATGATTGTTT,Pv@TTTATATTAT,Pm@TTGTATTAAT,Pow@ATTTACATAA,Poc@ATTTATATAT,Pk@TTTTTATTAT

sampleClass.locus.mito2.region=Pf_M76611:600-900 
sampleClass.locus.mito2.anchors=741@GAATAGAA...GAACTCTATAAATAACCA
sampleClass.locus.mito2.targets=species2@728-733&740-740&749-751&770-773
sampleClass.locus.mito2.target.species2.alleles=Pf@GTTCATTTAAGATT|GTTCATTTAAGACT,Pv|Pk@TATTCATAAATACA,Pm@GTTCAATTAGTACT,Pow|Poc@GTTACAATAATATT

sampleClass.locus.mito3.region=Pf_M76611:720-1020 
sampleClass.locus.mito3.anchors=842@(?:GAAAGAATTTATAA|ATATA[AG]TGAATATG)ACCAT
sampleClass.locus.mito3.targets=species3@861-869&878-881&884-887
sampleClass.locus.mito3.target.species3.alleles=Pf@TCGGTAGAATATTTATT,Pv@TCACTATTACATTAACT,Pm@TCACTATTTAATATATC,Pow@CCCTTATTTAACTAACC|TCCTTATTTAACTAACC,Poc@TCGTTATTAAACTAACC,Pk@TCACAATTAAACTTATT

sampleClass.locus.mito4.region=Pf_M76611:820-1120 
sampleClass.locus.mito4.anchors=948@CCTGTAACACAATAAAATAATGT
sampleClass.locus.mito4.targets=species4@971-982
sampleClass.locus.mito4.target.species4.alleles=Pf@AGTATATACAGT,Pv|Pow|Poc@ACCAGATATAGC,Pm@TCCTGAAACTCC,Pk@ACCTGATATAGC

sampleClass.locus.mito5.region=Pf_M76611:900-1200 
sampleClass.locus.mito5.anchors=1029@GATGCAAAACATTCTCC
sampleClass.locus.mito5.targets=species5@1025-1028&1046-1049
sampleClass.locus.mito5.target.species5.alleles=Pf@TAGATAAT,Pv|Pk@AAGTAAGT,Pm@TAATAAGT,Pow@TAATAAGA,Poc@TAATAAGG

sampleClass.locus.mito6.region=Pf_M76611:950-1250
sampleClass.locus.mito6.anchors=1077@ATTTC[AT]AAACTCAT[TA]CCTTTTTCTA
sampleClass.locus.mito6.targets=species6@1062-1066&1073-1073&1076-1076&1082-1082&1091-1091&1102-1108
sampleClass.locus.mito6.target.species6.alleles=Pf@CAAATAGATTAAATAC,Pv|Pk@AATACAATTTTAGAAA|AATATAATTTTAGAAA,Pm@AATATTTAAAAAGAAA,Pow|Poc@AATATTTTTTGAGAAA|AATATTTTTTAAGAAA
''', file=fo)
fo.close() 

In [36]:
fo = open(species_inc_unmapped_properties_fn, 'w')
print('''sampleClass.classes=Pf,Pv,Pm,Pow,Poc,Pk
sampleClass.loci=mito1,mito2,mito3,mito4,mito5,mito6 

sampleClass.locus.mito1.region=Pf_M76611:520-820 
sampleClass.locus.mito1.anchors=651@CCTTACGTACTCTAGCT....ACACAA
sampleClass.locus.mito1.targets=species1@668-671&678-683
sampleClass.locus.mito1.target.species1.alleles=Pf@ATGATTGTCT|ATGATTGTTT,Pv@TTTATATTAT,Pm@TTGTATTAAT,Pow@ATTTACATAA,Poc@ATTTATATAT,Pk@TTTTTATTAT
sampleClass.locus.mito1.analyzeUnmappedReads=true

sampleClass.locus.mito2.region=Pf_M76611:600-900 
sampleClass.locus.mito2.anchors=741@GAATAGAA...GAACTCTATAAATAACCA
sampleClass.locus.mito2.targets=species2@728-733&740-740&749-751&770-773
sampleClass.locus.mito2.target.species2.alleles=Pf@GTTCATTTAAGATT|GTTCATTTAAGACT,Pv|Pk@TATTCATAAATACA,Pm@GTTCAATTAGTACT,Pow|Poc@GTTACAATAATATT
sampleClass.locus.mito2.analyzeUnmappedReads=true

sampleClass.locus.mito3.region=Pf_M76611:720-1020 
sampleClass.locus.mito3.anchors=842@(?:GAAAGAATTTATAA|ATATA[AG]TGAATATG)ACCAT
sampleClass.locus.mito3.targets=species3@861-869&878-881&884-887
sampleClass.locus.mito3.target.species3.alleles=Pf@TCGGTAGAATATTTATT,Pv@TCACTATTACATTAACT,Pm@TCACTATTTAATATATC,Pow@CCCTTATTTAACTAACC|TCCTTATTTAACTAACC,Poc@TCGTTATTAAACTAACC,Pk@TCACAATTAAACTTATT
sampleClass.locus.mito3.analyzeUnmappedReads=true

sampleClass.locus.mito4.region=Pf_M76611:820-1120 
sampleClass.locus.mito4.anchors=948@CCTGTAACACAATAAAATAATGT
sampleClass.locus.mito4.targets=species4@971-982
sampleClass.locus.mito4.target.species4.alleles=Pf@AGTATATACAGT,Pv|Pow|Poc@ACCAGATATAGC,Pm@TCCTGAAACTCC,Pk@ACCTGATATAGC
sampleClass.locus.mito4.analyzeUnmappedReads=true

sampleClass.locus.mito5.region=Pf_M76611:900-1200 
sampleClass.locus.mito5.anchors=1029@GATGCAAAACATTCTCC
sampleClass.locus.mito5.targets=species5@1025-1028&1046-1049
sampleClass.locus.mito5.target.species5.alleles=Pf@TAGATAAT,Pv|Pk@AAGTAAGT,Pm@TAATAAGT,Pow@TAATAAGA,Poc@TAATAAGG
sampleClass.locus.mito5.analyzeUnmappedReads=true

sampleClass.locus.mito6.region=Pf_M76611:950-1250
sampleClass.locus.mito6.anchors=1077@ATTTC[AT]AAACTCAT[TA]CCTTTTTCTA
sampleClass.locus.mito6.targets=species6@1062-1066&1073-1073&1076-1076&1082-1082&1091-1091&1102-1108
sampleClass.locus.mito6.target.species6.alleles=Pf@CAAATAGATTAAATAC,Pv|Pk@AATACAATTTTAGAAA|AATATAATTTTAGAAA,Pm@AATATTTAAAAAGAAA,Pow|Poc@AATATTTTTTGAGAAA|AATATTTTTTAAGAAA
sampleClass.locus.mito6.analyzeUnmappedReads=true
''', file=fo)
fo.close() 

In [21]:
fo = open(chromosomeMap_fn, 'w')
print('''default
Pf3D7_01_v3
Pf3D7_02_v3
Pf3D7_03_v3
Pf3D7_04_v3
Pf3D7_05_v3
Pf3D7_06_v3
Pf3D7_07_v3
Pf3D7_08_v3
Pf3D7_09_v3
Pf3D7_10_v3
Pf3D7_11_v3
Pf3D7_12_v3
Pf3D7_13_v3
Pf3D7_14_v3
Pf_M76611
Pf3D7_API_v3''', file=fo)
fo.close()


# Pipeline scripts

In [22]:
fo = open(runArrayJob_fn, 'w')
print('''BAMLIST_FILE=$1
CONFIG_FILE=$2
REF_FASTA_FILE=$3
CHR_MAP_FILE=$4
OUT_DIR=$5
 
JOB=$LSB_JOBINDEX
#JOB=3
 
IN=`sed "$JOB q;d" $BAMLIST_FILE`
read -a LINE <<< "$IN"
SAMPLE_NAME=${LINE[0]}
BAM_FILE=${LINE[1]}
CHR_MAP_NAME=${LINE[2]}
 
JAVA_HOME=/nfs/team112_internal/rp7/opt/java/jdk1.8.0_101
JRE_HOME=/nfs/team112_internal/rp7/opt/java/jdk1.8.0_101

GRCC=/nfs/team112_internal/rp7/src/github/malariagen/GeneticReportCard/AnalysisCommon
GRCA=/nfs/team112_internal/rp7/src/github/malariagen/GeneticReportCard/SequencingReadsAnalysis
CLASSPATH=$GRCA/bin:$GRCC/bin:\
$GRCC/lib/commons-logging-1.1.1.jar:\
$GRCA/lib/apache-ant-1.8.2-bzip2.jar:\
$GRCA/lib/commons-compress-1.4.1.jar:\
$GRCA/lib/commons-jexl-2.1.1.jar:\
$GRCA/lib/htsjdk-2.1.0.jar:\
$GRCA/lib/ngs-java-1.2.2.jar:\
$GRCA/lib/snappy-java-1.0.3-rc3.jar:\
$GRCA/lib/xz-1.5.jar

$JAVA_HOME/bin/java -cp $CLASSPATH -Xms512m -Xmx2000m 'org.cggh.bam.grc.GrcAnalysis$SingleSample' $CONFIG_FILE $SAMPLE_NAME $BAM_FILE $CHR_MAP_NAME $REF_FASTA_FILE $CHR_MAP_FILE $OUT_DIR
''', file=fo)
fo.close()



In [23]:
fo = open(runSpeciesArrayJob_fn, 'w')
print('''BAMLIST_FILE=$1
CONFIG_FILE=$2
REF_FASTA_FILE=$3
CHR_MAP_FILE=$4
OUT_DIR=$5
 
JOB=$LSB_JOBINDEX
#JOB=3
 
IN=`sed "$JOB q;d" $BAMLIST_FILE`
read -a LINE <<< "$IN"
SAMPLE_NAME=${LINE[0]}
BAM_FILE=${LINE[1]}
CHR_MAP_NAME=${LINE[2]}
 
JAVA_HOME=/nfs/team112_internal/rp7/opt/java/jdk1.8.0_101
JRE_HOME=/nfs/team112_internal/rp7/opt/java/jdk1.8.0_101

GRCC=/nfs/team112_internal/rp7/src/github/malariagen/GeneticReportCard/AnalysisCommon
GRCA=/nfs/team112_internal/rp7/src/github/malariagen/GeneticReportCard/SequencingReadsAnalysis
CLASSPATH=$GRCA/bin:$GRCC/bin:\
$GRCC/lib/commons-logging-1.1.1.jar:\
$GRCA/lib/apache-ant-1.8.2-bzip2.jar:\
$GRCA/lib/commons-compress-1.4.1.jar:\
$GRCA/lib/commons-jexl-2.1.1.jar:\
$GRCA/lib/htsjdk-2.1.0.jar:\
$GRCA/lib/ngs-java-1.2.2.jar:\
$GRCA/lib/snappy-java-1.0.3-rc3.jar:\
$GRCA/lib/xz-1.5.jar

$JAVA_HOME/bin/java -cp $CLASSPATH -Xms512m -Xmx12000m 'org.cggh.bam.sampleClass.SampleClassAnalysis$SingleSample' $CONFIG_FILE $SAMPLE_NAME $BAM_FILE $CHR_MAP_NAME $REF_FASTA_FILE $CHR_MAP_FILE $OUT_DIR
''', file=fo)
fo.close()



In [24]:
fo = open(submitArray_fn, 'w')
print('''BAMLIST_FILE=%s
CONFIG_FILE=%s
REF_FASTA_FILE=%s
CHR_MAP_FILE=%s
OUT_DIR=%s/grc
LOG_DIR=%s/log
 
NUM_BAMLIST_LINES=`wc -l < $BAMLIST_FILE`
QUEUE=normal
# NUM_BAMLIST_LINES=2
# QUEUE=small

bsub -q $QUEUE -G malaria-dk -J "genotype[2-$NUM_BAMLIST_LINES]%%1000" -R"select[mem>2000] rusage[mem=2000] span[hosts=1]" -M 2000 -o $LOG_DIR/output_%%J-%%I.log %s $BAMLIST_FILE $CONFIG_FILE $REF_FASTA_FILE $CHR_MAP_FILE $OUT_DIR
''' % (
        bam_list_fn,
        grc_properties_fn,
        ref_fasta_fn,
        chromosomeMap_fn,
        scratch_dir,
        scratch_dir,
        "bash %s" % runArrayJob_fn,
        ),
     file=fo)
fo.close()



In [41]:
fo = open(submitIncUnmappedArray_fn, 'w')
print('''BAMLIST_FILE=%s
CONFIG_FILE=%s
REF_FASTA_FILE=%s
CHR_MAP_FILE=%s
OUT_DIR=%s/grc_inc_unmapped
LOG_DIR=%s/log
 
NUM_BAMLIST_LINES=`wc -l < $BAMLIST_FILE`
QUEUE=normal
# NUM_BAMLIST_LINES=2
# QUEUE=small

bsub -q $QUEUE -G malaria-dk -J "genotype[2-$NUM_BAMLIST_LINES]%%1000" -R"select[mem>2000] rusage[mem=2000] span[hosts=1]" -M 2000 -o $LOG_DIR/output_%%J-%%I.log %s $BAMLIST_FILE $CONFIG_FILE $REF_FASTA_FILE $CHR_MAP_FILE $OUT_DIR
''' % (
        bam_list_fn,
        grc_inc_unmapped_properties_fn,
        ref_fasta_fn,
        chromosomeMap_fn,
        scratch_dir,
        scratch_dir,
        "bash %s" % runArrayJob_fn,
        ),
     file=fo)
fo.close()



In [25]:
fo = open(submitSpeciesArray_fn, 'w')
print('''BAMLIST_FILE=%s
CONFIG_FILE=%s
REF_FASTA_FILE=%s
CHR_MAP_FILE=%s
OUT_DIR=%s/species
LOG_DIR=%s/log
 
NUM_BAMLIST_LINES=`wc -l < $BAMLIST_FILE`
QUEUE=normal
# NUM_BAMLIST_LINES=2
# QUEUE=small

bsub -q $QUEUE -G malaria-dk -J "genotype[2-$NUM_BAMLIST_LINES]%%1000" -n9 -R"select[mem>16000] rusage[mem=16000] span[hosts=1]" -M 16000 -o $LOG_DIR/output_%%J-%%I.log %s $BAMLIST_FILE $CONFIG_FILE $REF_FASTA_FILE $CHR_MAP_FILE $OUT_DIR
''' % (
        bam_list_fn,
        species_properties_fn,
        ref_fasta_fn,
        chromosomeMap_fn,
        scratch_dir,
        scratch_dir,
        "bash %s" % runSpeciesArrayJob_fn,
        ),
     file=fo)
fo.close()



In [26]:
fo = open(submitSpeciesIncUnmappedArray_fn, 'w')
print('''BAMLIST_FILE=%s
CONFIG_FILE=%s
REF_FASTA_FILE=%s
CHR_MAP_FILE=%s
OUT_DIR=%s/species_inc_unmapped
LOG_DIR=%s/log
 
NUM_BAMLIST_LINES=`wc -l < $BAMLIST_FILE`
QUEUE=normal
# NUM_BAMLIST_LINES=2
# QUEUE=small

bsub -q $QUEUE -G malaria-dk -J "genotype[2-$NUM_BAMLIST_LINES]%%1000" -n9 -R"select[mem>16000] rusage[mem=16000] span[hosts=1]" -M 16000 -o $LOG_DIR/output_%%J-%%I.log %s $BAMLIST_FILE $CONFIG_FILE $REF_FASTA_FILE $CHR_MAP_FILE $OUT_DIR
''' % (
        bam_list_fn,
        species_inc_unmapped_properties_fn,
        ref_fasta_fn,
        chromosomeMap_fn,
        scratch_dir,
        scratch_dir,
        "bash %s" % runSpeciesArrayJob_fn,
        ),
     file=fo)
fo.close()



In [27]:
fo = open(mergeGrcResults_fn, 'w')
print('''BAMLIST_FILE=%s
CONFIG_FILE=%s
REF_FASTA_FILE=%s
CHR_MAP_FILE=%s
OUT_DIR=%s/grc
 
JAVA_HOME=/nfs/team112_internal/rp7/opt/java/jdk1.8.0_101
JRE_HOME=/nfs/team112_internal/rp7/opt/java/jdk1.8.0_101

GRCC=/nfs/team112_internal/rp7/src/github/malariagen/GeneticReportCard/AnalysisCommon
GRCA=/nfs/team112_internal/rp7/src/github/malariagen/GeneticReportCard/SequencingReadsAnalysis
CLASSPATH=$GRCA/bin:$GRCC/bin:\
$GRCC/lib/commons-logging-1.1.1.jar:\
$GRCA/lib/apache-ant-1.8.2-bzip2.jar:\
$GRCA/lib/commons-compress-1.4.1.jar:\
$GRCA/lib/commons-jexl-2.1.1.jar:\
$GRCA/lib/htsjdk-2.1.0.jar:\
$GRCA/lib/ngs-java-1.2.2.jar:\
$GRCA/lib/snappy-java-1.0.3-rc3.jar:\
$GRCA/lib/xz-1.5.jar

$JAVA_HOME/bin/java -cp $CLASSPATH -Xms512m -Xmx2000m 'org.cggh.bam.grc.GrcAnalysis$MergeResults' $CONFIG_FILE $BAMLIST_FILE $REF_FASTA_FILE $CHR_MAP_FILE $OUT_DIR
''' % (
        bam_list_fn,
        grc_properties_fn,
        ref_fasta_fn,
        chromosomeMap_fn,
        scratch_dir,
        ),
     file=fo)
fo.close()

In [42]:
fo = open(mergeGrcIncUnmappedResults_fn, 'w')
print('''BAMLIST_FILE=%s
CONFIG_FILE=%s
REF_FASTA_FILE=%s
CHR_MAP_FILE=%s
OUT_DIR=%s/grc_inc_unmapped
 
JAVA_HOME=/nfs/team112_internal/rp7/opt/java/jdk1.8.0_101
JRE_HOME=/nfs/team112_internal/rp7/opt/java/jdk1.8.0_101

GRCC=/nfs/team112_internal/rp7/src/github/malariagen/GeneticReportCard/AnalysisCommon
GRCA=/nfs/team112_internal/rp7/src/github/malariagen/GeneticReportCard/SequencingReadsAnalysis
CLASSPATH=$GRCA/bin:$GRCC/bin:\
$GRCC/lib/commons-logging-1.1.1.jar:\
$GRCA/lib/apache-ant-1.8.2-bzip2.jar:\
$GRCA/lib/commons-compress-1.4.1.jar:\
$GRCA/lib/commons-jexl-2.1.1.jar:\
$GRCA/lib/htsjdk-2.1.0.jar:\
$GRCA/lib/ngs-java-1.2.2.jar:\
$GRCA/lib/snappy-java-1.0.3-rc3.jar:\
$GRCA/lib/xz-1.5.jar

$JAVA_HOME/bin/java -cp $CLASSPATH -Xms512m -Xmx2000m 'org.cggh.bam.grc.GrcAnalysis$MergeResults' $CONFIG_FILE $BAMLIST_FILE $REF_FASTA_FILE $CHR_MAP_FILE $OUT_DIR
''' % (
        bam_list_fn,
        grc_inc_unmapped_properties_fn,
        ref_fasta_fn,
        chromosomeMap_fn,
        scratch_dir,
        ),
     file=fo)
fo.close()

In [28]:
fo = open(mergeSpeciesResults_fn, 'w')
print('''BAMLIST_FILE=%s
CONFIG_FILE=%s
REF_FASTA_FILE=%s
CHR_MAP_FILE=%s
OUT_DIR=%s/species
 
JAVA_HOME=/nfs/team112_internal/rp7/opt/java/jdk1.8.0_101
JRE_HOME=/nfs/team112_internal/rp7/opt/java/jdk1.8.0_101

GRCC=/nfs/team112_internal/rp7/src/github/malariagen/GeneticReportCard/AnalysisCommon
GRCA=/nfs/team112_internal/rp7/src/github/malariagen/GeneticReportCard/SequencingReadsAnalysis
CLASSPATH=$GRCA/bin:$GRCC/bin:\
$GRCC/lib/commons-logging-1.1.1.jar:\
$GRCA/lib/apache-ant-1.8.2-bzip2.jar:\
$GRCA/lib/commons-compress-1.4.1.jar:\
$GRCA/lib/commons-jexl-2.1.1.jar:\
$GRCA/lib/htsjdk-2.1.0.jar:\
$GRCA/lib/ngs-java-1.2.2.jar:\
$GRCA/lib/snappy-java-1.0.3-rc3.jar:\
$GRCA/lib/xz-1.5.jar

$JAVA_HOME/bin/java -cp $CLASSPATH -Xms512m -Xmx2000m 'org.cggh.bam.sampleClass.SampleClassAnalysis$MergeResults' $CONFIG_FILE $BAMLIST_FILE $REF_FASTA_FILE $CHR_MAP_FILE $OUT_DIR
''' % (
        bam_list_fn,
        species_properties_fn,
        ref_fasta_fn,
        chromosomeMap_fn,
        scratch_dir,
        ),
     file=fo)
fo.close()

In [29]:
fo = open(mergeSpeciesIncUnmappedResults_fn, 'w')
print('''BAMLIST_FILE=%s
CONFIG_FILE=%s
REF_FASTA_FILE=%s
CHR_MAP_FILE=%s
OUT_DIR=%s/species_inc_unmapped
 
JAVA_HOME=/nfs/team112_internal/rp7/opt/java/jdk1.8.0_101
JRE_HOME=/nfs/team112_internal/rp7/opt/java/jdk1.8.0_101

GRCC=/nfs/team112_internal/rp7/src/github/malariagen/GeneticReportCard/AnalysisCommon
GRCA=/nfs/team112_internal/rp7/src/github/malariagen/GeneticReportCard/SequencingReadsAnalysis
CLASSPATH=$GRCA/bin:$GRCC/bin:\
$GRCC/lib/commons-logging-1.1.1.jar:\
$GRCA/lib/apache-ant-1.8.2-bzip2.jar:\
$GRCA/lib/commons-compress-1.4.1.jar:\
$GRCA/lib/commons-jexl-2.1.1.jar:\
$GRCA/lib/htsjdk-2.1.0.jar:\
$GRCA/lib/ngs-java-1.2.2.jar:\
$GRCA/lib/snappy-java-1.0.3-rc3.jar:\
$GRCA/lib/xz-1.5.jar

$JAVA_HOME/bin/java -cp $CLASSPATH -Xms512m -Xmx2000m 'org.cggh.bam.sampleClass.SampleClassAnalysis$MergeResults' $CONFIG_FILE $BAMLIST_FILE $REF_FASTA_FILE $CHR_MAP_FILE $OUT_DIR
''' % (
        bam_list_fn,
        species_inc_unmapped_properties_fn,
        ref_fasta_fn,
        chromosomeMap_fn,
        scratch_dir,
        ),
     file=fo)
fo.close()

# Kicking off pipeline - species

In [31]:
!bash {submitSpeciesArray_fn}

Job <3423475> is submitted to queue <normal>.


In [33]:
!bash {mergeSpeciesResults_fn}

Mar 03, 2017 11:33:39 AM org.cggh.bam.sampleClass.SampleClassAnalysis$MergeResults main
INFO: ConfigFile: /nfs/team112_internal/rp7/data/methods-dev/builds/Pf6.0/20170303_run_Olivo_GRC/species/species.properties
Mar 03, 2017 11:33:39 AM org.cggh.bam.sampleClass.SampleClassAnalysis$MergeResults main
INFO: SampleListFile: /nfs/team112_internal/rp7/data/methods-dev/builds/Pf6.0/20170303_run_Olivo_GRC/pf_60_mergelanes_bamfiles.txt
Mar 03, 2017 11:33:39 AM org.cggh.bam.sampleClass.SampleClassAnalysis$MergeResults main
INFO: RefFastaFile: /nfs/team112_internal/rp7/data/methods-dev/builds/Pf6.0/20170303_run_Olivo_GRC/Pfalciparum.genome.fasta
Mar 03, 2017 11:33:39 AM org.cggh.bam.sampleClass.SampleClassAnalysis$MergeResults main
INFO: ChrMapFile: /nfs/team112_internal/rp7/data/methods-dev/builds/Pf6.0/20170303_run_Olivo_GRC/chromosomeMap.tab
Mar 03, 2017 11:33:39 AM org.cggh.bam.sampleClass.SampleClassAnalysis$MergeResults main
INFO: RootFolder: /lustre/scratch118/malaria/team112/personal/rp7/

In [37]:
!bash {submitSpeciesIncUnmappedArray_fn}

Job <3424608> is submitted to queue <normal>.


In [49]:
!bash {mergeSpeciesIncUnmappedResults_fn}

Mar 03, 2017 3:22:08 PM org.cggh.bam.sampleClass.SampleClassAnalysis$MergeResults main
INFO: ConfigFile: /nfs/team112_internal/rp7/data/methods-dev/builds/Pf6.0/20170303_run_Olivo_GRC/species_inc_unmapped/species.properties
Mar 03, 2017 3:22:08 PM org.cggh.bam.sampleClass.SampleClassAnalysis$MergeResults main
INFO: SampleListFile: /nfs/team112_internal/rp7/data/methods-dev/builds/Pf6.0/20170303_run_Olivo_GRC/pf_60_mergelanes_bamfiles.txt
Mar 03, 2017 3:22:08 PM org.cggh.bam.sampleClass.SampleClassAnalysis$MergeResults main
INFO: RefFastaFile: /nfs/team112_internal/rp7/data/methods-dev/builds/Pf6.0/20170303_run_Olivo_GRC/Pfalciparum.genome.fasta
Mar 03, 2017 3:22:08 PM org.cggh.bam.sampleClass.SampleClassAnalysis$MergeResults main
INFO: ChrMapFile: /nfs/team112_internal/rp7/data/methods-dev/builds/Pf6.0/20170303_run_Olivo_GRC/chromosomeMap.tab
Mar 03, 2017 3:22:08 PM org.cggh.bam.sampleClass.SampleClassAnalysis$MergeResults main
INFO: RootFolder: /lustre/scratch118/malaria/team112/perso

# Kicking off pipeline - GRC

In [32]:
!bash {submitArray_fn}

Job <3423537> is submitted to queue <normal>.


In [35]:
!bash {mergeGrcResults_fn}

Mar 03, 2017 11:35:37 AM org.cggh.bam.grc.GrcAnalysis$MergeResults main
INFO: ConfigFile: /nfs/team112_internal/rp7/data/methods-dev/builds/Pf6.0/20170303_run_Olivo_GRC/grc/grc.properties
Mar 03, 2017 11:35:37 AM org.cggh.bam.grc.GrcAnalysis$MergeResults main
INFO: SampleListFile: /nfs/team112_internal/rp7/data/methods-dev/builds/Pf6.0/20170303_run_Olivo_GRC/pf_60_mergelanes_bamfiles.txt
Mar 03, 2017 11:35:37 AM org.cggh.bam.grc.GrcAnalysis$MergeResults main
INFO: RefFastaFile: /nfs/team112_internal/rp7/data/methods-dev/builds/Pf6.0/20170303_run_Olivo_GRC/Pfalciparum.genome.fasta
Mar 03, 2017 11:35:37 AM org.cggh.bam.grc.GrcAnalysis$MergeResults main
INFO: ChrMapFile: /nfs/team112_internal/rp7/data/methods-dev/builds/Pf6.0/20170303_run_Olivo_GRC/chromosomeMap.tab
Mar 03, 2017 11:35:37 AM org.cggh.bam.grc.GrcAnalysis$MergeResults main
INFO: RootFolder: /lustre/scratch118/malaria/team112/personal/rp7/data/methods-dev/builds/Pf6.0/20170303_run_Olivo_GRC/grc
Mar 03, 2017 11:35:38 AM org.cg

In [43]:
!bash {submitIncUnmappedArray_fn}

User <rp7>: Pending job threshold reached. Retrying in 60 seconds...
User <rp7>: Pending job threshold reached. Retrying in 60 seconds...
User <rp7>: Pending job threshold reached. Retrying in 60 seconds...
User <rp7>: Pending job threshold reached. Retrying in 60 seconds...
Job <3425348> is submitted to queue <normal>.


In [35]:
!bash {mergeGrcIncUnmappedResults_fn}

Mar 03, 2017 11:35:37 AM org.cggh.bam.grc.GrcAnalysis$MergeResults main
INFO: ConfigFile: /nfs/team112_internal/rp7/data/methods-dev/builds/Pf6.0/20170303_run_Olivo_GRC/grc/grc.properties
Mar 03, 2017 11:35:37 AM org.cggh.bam.grc.GrcAnalysis$MergeResults main
INFO: SampleListFile: /nfs/team112_internal/rp7/data/methods-dev/builds/Pf6.0/20170303_run_Olivo_GRC/pf_60_mergelanes_bamfiles.txt
Mar 03, 2017 11:35:37 AM org.cggh.bam.grc.GrcAnalysis$MergeResults main
INFO: RefFastaFile: /nfs/team112_internal/rp7/data/methods-dev/builds/Pf6.0/20170303_run_Olivo_GRC/Pfalciparum.genome.fasta
Mar 03, 2017 11:35:37 AM org.cggh.bam.grc.GrcAnalysis$MergeResults main
INFO: ChrMapFile: /nfs/team112_internal/rp7/data/methods-dev/builds/Pf6.0/20170303_run_Olivo_GRC/chromosomeMap.tab
Mar 03, 2017 11:35:37 AM org.cggh.bam.grc.GrcAnalysis$MergeResults main
INFO: RootFolder: /lustre/scratch118/malaria/team112/personal/rp7/data/methods-dev/builds/Pf6.0/20170303_run_Olivo_GRC/grc
Mar 03, 2017 11:35:38 AM org.cg

# Debugging

## PF0810-C
This sample previously called Pf,Poc and Pf,Po in Olivo and Chris's files respectively. Not called in current

In [45]:
!cat {scratch_dir}/species/PF08/PF0810-C.classAlleles.tab

Num	Sample	Locus	Target	Allele	Count	SampleClass
1	PF0810-C	mito1	species1	Pf	0	-
2	PF0810-C	mito1	species1	Pv	0	-
3	PF0810-C	mito1	species1	Pm	0	-
4	PF0810-C	mito1	species1	Pow	0	-
5	PF0810-C	mito1	species1	Poc	0	-
6	PF0810-C	mito1	species1	Pk	0	-
7	PF0810-C	mito2	species2	Pf	0	-
8	PF0810-C	mito2	species2	Pv|Pk	0	-
9	PF0810-C	mito2	species2	Pm	0	-
10	PF0810-C	mito2	species2	Pow|Poc	0	-
11	PF0810-C	mito3	species3	Pf	0	-
12	PF0810-C	mito3	species3	Pv	0	-
13	PF0810-C	mito3	species3	Pm	0	-
14	PF0810-C	mito3	species3	Pow	0	-
15	PF0810-C	mito3	species3	Poc	0	-
16	PF0810-C	mito3	species3	Pk	0	-
17	PF0810-C	mito4	species4	Pf	0	-
18	PF0810-C	mito4	species4	Pv|Pow|Poc	0	-
19	PF0810-C	mito4	species4	Pm	0	-
20	PF0810-C	mito4	species4	Pk	0	-
21	PF0810-C	mito5	species5	Pf	0	-
22	PF0810-C	mito5	species5	Pv|Pk	0	-
23	PF0810-C	mito5	species5	Pm	0	-
24	PF0810-C	mito5	species5	Pow	0	-
25	PF0810-C	mito5	species5	Poc	0	-
26	PF0810-C	mito6	species6	Pf	0	-
27	PF0810-C	mito6	species6	Pv|Pk	0	-
28	PF0810-C	mi

In [48]:
!cat {scratch_dir}/species_inc_unmapped/PF08/PF0810-C.classAlleles.tab

Num	Sample	Locus	Target	Allele	Count	SampleClass
1	PF0810-C	mito1	species1	Pf	0	-
2	PF0810-C	mito1	species1	Pv	0	-
3	PF0810-C	mito1	species1	Pm	0	-
4	PF0810-C	mito1	species1	Pow	0	-
5	PF0810-C	mito1	species1	Poc	0	-
6	PF0810-C	mito1	species1	Pk	0	-
7	PF0810-C	mito2	species2	Pf	0	-
8	PF0810-C	mito2	species2	Pv|Pk	0	-
9	PF0810-C	mito2	species2	Pm	0	-
10	PF0810-C	mito2	species2	Pow|Poc	0	-
11	PF0810-C	mito3	species3	Pf	0	-
12	PF0810-C	mito3	species3	Pv	0	-
13	PF0810-C	mito3	species3	Pm	0	-
14	PF0810-C	mito3	species3	Pow	0	-
15	PF0810-C	mito3	species3	Poc	0	-
16	PF0810-C	mito3	species3	Pk	0	-
17	PF0810-C	mito4	species4	Pf	0	-
18	PF0810-C	mito4	species4	Pv|Pow|Poc	0	-
19	PF0810-C	mito4	species4	Pm	0	-
20	PF0810-C	mito4	species4	Pk	0	-
21	PF0810-C	mito5	species5	Pf	0	-
22	PF0810-C	mito5	species5	Pv|Pk	0	-
23	PF0810-C	mito5	species5	Pm	0	-
24	PF0810-C	mito5	species5	Pow	0	-
25	PF0810-C	mito5	species5	Poc	0	-
26	PF0810-C	mito6	species6	Pf	0	-
27	PF0810-C	mito6	species6	Pv|Pk	0	-
28	PF0810-C	mi

# Copy final files to nfs

In [51]:
!cp -R {scratch_dir}/species {output_dir}
!cp -R {scratch_dir}/species_inc_unmapped {output_dir}
!cp -R {scratch_dir}/grc {output_dir}
!cp -R {scratch_dir}/grc_inc_unmapped {output_dir}


In [59]:
output_dir

'/nfs/team112_internal/rp7/data/methods-dev/builds/Pf6.0/20170302_run_Olivo_GRC'

# Warning - some samples appear multiple times!
This was based on an earlier run that include single-end bam files. After re-running we see no such problem.

In [60]:
tbl_pf60_species = (
    etl
    .fromtsv("%s/species/AllSamples-AllTargets.classes.tab" % scratch_dir)
    .cut(['Sample', 'SampleClass', 'Consensus'])
)
print(len(tbl_pf60_species.data()))
print(len(tbl_pf60_species.unique('Sample').data()))
tbl_pf60_species


8251
8243


Sample,SampleClass,Consensus
PA0867-CW,-,Y
PG0035-Cx,Pf,Y
PF0252-C,Pf,Y
PG0528-C,Pf,Y
PM0603-C,Pf,Y


In [61]:
tbl_pf60_species.duplicates('Sample').displayall()

Sample,SampleClass,Consensus
PG0002-C,Pf,Y
PG0002-C,Pf,Y
PM0006-C,Pf,Y
PM0006-C,Pf,Y
PM0007-C,Pf,Y
PM0007-C,Pf,Y
PM0008-C,Pf,Y
PM0008-C,Pf,Y


In [62]:
!grep PG0002 {scratch_dir}/species/AllSamples-AllTargets.classes.tab

26	PG0002-C	Pf	Y	Pf	Pf	Pf	Pf	Pf	Pf
27	PG0002-C	Pf	Y	Pf	Pf	Pf	Pf	Pf	Pf


In [63]:
!grep PM000 {scratch_dir}/species/AllSamples-AllTargets.classes.tab

252	PM0006-C	Pf	Y	Pf	Pf	Pf	Pf	Pf	Pf
253	PM0006-C	Pf	Y	Pf	Pf	Pf	Pf	Pf	Pf
366	PM0007-C	Pf	Y	Pf	-	Pf	Pf	Pf	-
367	PM0007-C	Pf	Y	Pf	-	Pf	Pf	Pf	-
418	PM0008-C	Pf	Y	Pf	-	Pf	Pf	Pf	-
419	PM0008-C	Pf	Y	Pf	-	Pf	Pf	Pf	-
1507	PM0009-C	Pf	Y	Pf	-	Pf	Pf	Pf	-
3316	PM0002-C	Pf	Y	Pf	-	Pf	Pf	Pf	-
4028	PM0003-C	Pf	Y	Pf	-	Pf	Pf	Pf	-
4239	PM0005-C	Pf	Y	Pf	-	Pf	Pf	Pf	-
5645	PM0001-C	Pf	Y	Pf	-	Pf	Pf	Pf	-
7142	PM0004-C	Pf	Y	Pf	Pf	Pf	Pf	Pf	Pf


# Some error checking from previous run - not needed this time

In [56]:
!wc -l {scratch_dir}/species/AllSamples-AllTargets.classes.tab

8251 /lustre/scratch118/malaria/team112/personal/rp7/data/methods-dev/builds/Pf6.0/20170302_run_Olivo_GRC/species/AllSamples-AllTargets.classes.tab


In [55]:
!wc -l {scratch_dir}/grc/AllCallsBySample.tab

8251 /lustre/scratch118/malaria/team112/personal/rp7/data/methods-dev/builds/Pf6.0/20170302_run_Olivo_GRC/grc/AllCallsBySample.tab


In [48]:
!ls -altr {scratch_dir}/species/*/*.classAlleles.tab | wc -l

8247


In [44]:
!ls -altr {scratch_dir}/species/*/*.unlistedAlleles.tab | wc -l

7724


In [52]:
!ls -altr {scratch_dir}/grc/*/*.locusCoverage.tab | wc -l

8247


In [53]:
!ls -altr {scratch_dir}/grc/*/*.alleles.tab | wc -l

8164


In [47]:
# Which jobs failed in the above?
!grep -i 'error\|memlimit' /lustre/scratch118/malaria/team112/personal/rp7/data/methods-dev/builds/Pf6.0/20170302_run_Olivo_GRC/log/output_3420683*

In [16]:
!sed "7743 q;d" {bam_list_fn}

PA0321-CW	/lustre/scratch116/malaria/pfalciparum/output/8/0/d/d/37201/4_bam_mark_duplicates_v2/pe.1.markdup.bam	default


In [4]:
!grep -nr PA0312 {bam_list_fn}

390:PA0312-CW	/lustre/scratch116/malaria/pfalciparum/output/3/8/1/0/37197/4_bam_mark_duplicates_v2/pe.1.markdup.bam	default


In [44]:
!grep -nr PA0867 {bam_list_fn}

2:PA0867-CW	/lustre/scratch116/malaria/pfalciparum/output/6/d/b/0/37432/4_bam_mark_duplicates_v2/pe.1.markdup.bam	default
